# Map reduce

The map reduce documents chain first applies an LLM chain to each document individually (the Map step), treating the chain output as a new document. It then passes all the new documents to a separate combine documents chain to get a single output (the Reduce step). It can optionally first compress, or collapse, the mapped documents to make sure that they fit in the combine documents chain (which will often pass them to an LLM). This compression step is performed recursively if necessary.

![map_reduce_diagram](/img/map_reduce.jpg)

## Recreating with LCEL

With [LangChain Expression Language](/docs/expression_language) we can recreate the `MapReduceDocumentsChain` functionality, with the additional benefit of getting all the built-in LCEL features (batch, async, etc.) and with much more ability to customize specific parts of the chain.

In [71]:
from functools import partial

from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.chains.combine_documents.reduce import _collapse_docs, _split_list_of_docs
from langchain.chat_models import ChatAnthropic
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

llm = ChatAnthropic()

document_prompt = PromptTemplate.from_template("{page_content}")
map_prompt = PromptTemplate.from_template("Summarize this content:\n\n{context}")
collapse_prompt = PromptTemplate.from_template("Collapse this content:\n\n{context}")
reduce_prompt = PromptTemplate.from_template("Combine these summaries:\n\n{context}")

partial_format_document = partial(format_document, prompt=document_prompt)
map_chain = (
    {"context": partial_format_document} 
    | map_prompt 
    | llm 
    | StrOutputParser()
)
map_as_doc_chain = (
    RunnableParallel({"doc": RunnablePassthrough(), "content": map_chain}) 
    | (lambda x: Document(page_content=x["content"], metadata=x["doc"].metadata))
).with_config(run_name="Summarize (return doc)")

def format_docs(docs):
    return "\n\n".join(partial_format_document(doc) for doc in docs)
    
collapse_chain = (
    {"context": format_docs}
    | collapse_prompt
    | llm
    | StrOutputParser()
)

def get_num_tokens(docs):
    return llm.get_num_tokens(format_docs(docs))

def collapse(
    docs, 
    config: RunnableConfig,
    run_manager: CallbackManagerForChainRun,
    token_max=4000,
):
    config["callbacks"] = run_manager.get_child()
    collapse_ct = 1
    while get_num_tokens(docs) > token_max:
        config["run_name"] = f"Collapse {collapse_ct}"
        invoke = partial(collapse_chain.invoke, config=config)
        split_docs = _split_list_of_docs(docs, get_num_tokens, token_max)
        docs = [_collapse_docs(_docs, invoke) for _docs in split_docs]
        collapse_ct += 1
    return docs

reduce_chain = (
    {"context": format_docs}
    | reduce_prompt
    | llm
    | StrOutputParser()
).with_config(run_name="Reduce")

map_reduce = (map_as_doc_chain.map() | collapse | reduce_chain).with_config(run_name="Map reduce")

## Example run

In [65]:
from langchain.schema import Document

text = """Nuclear power in space is the use of nuclear power in outer space, typically either small fission systems or radioactive decay for electricity or heat. Another use is for scientific observation, as in a Mössbauer spectrometer. The most common type is a radioisotope thermoelectric generator, which has been used on many space probes and on crewed lunar missions. Small fission reactors for Earth observation satellites, such as the TOPAZ nuclear reactor, have also been flown.[1] A radioisotope heater unit is powered by radioactive decay and can keep components from becoming too cold to function, potentially over a span of decades.[2]

The United States tested the SNAP-10A nuclear reactor in space for 43 days in 1965,[3] with the next test of a nuclear reactor power system intended for space use occurring on 13 September 2012 with the Demonstration Using Flattop Fission (DUFF) test of the Kilopower reactor.[4]

After a ground-based test of the experimental 1965 Romashka reactor, which used uranium and direct thermoelectric conversion to electricity,[5] the USSR sent about 40 nuclear-electric satellites into space, mostly powered by the BES-5 reactor. The more powerful TOPAZ-II reactor produced 10 kilowatts of electricity.[3]

Examples of concepts that use nuclear power for space propulsion systems include the nuclear electric rocket (nuclear powered ion thruster(s)), the radioisotope rocket, and radioisotope electric propulsion (REP).[6] One of the more explored concepts is the nuclear thermal rocket, which was ground tested in the NERVA program. Nuclear pulse propulsion was the subject of Project Orion.[7]

Regulation and hazard prevention[edit]
After the ban of nuclear weapons in space by the Outer Space Treaty in 1967, nuclear power has been discussed at least since 1972 as a sensitive issue by states.[8] Particularly its potential hazards to Earth's environment and thus also humans has prompted states to adopt in the U.N. General Assembly the Principles Relevant to the Use of Nuclear Power Sources in Outer Space (1992), particularly introducing safety principles for launches and to manage their traffic.[8]

Benefits

Both the Viking 1 and Viking 2 landers used RTGs for power on the surface of Mars. (Viking launch vehicle pictured)
While solar power is much more commonly used, nuclear power can offer advantages in some areas. Solar cells, although efficient, can only supply energy to spacecraft in orbits where the solar flux is sufficiently high, such as low Earth orbit and interplanetary destinations close enough to the Sun. Unlike solar cells, nuclear power systems function independently of sunlight, which is necessary for deep space exploration. Nuclear-based systems can have less mass than solar cells of equivalent power, allowing more compact spacecraft that are easier to orient and direct in space. In the case of crewed spaceflight, nuclear power concepts that can power both life support and propulsion systems may reduce both cost and flight time.[9]

Selected applications and/or technologies for space include:

Radioisotope thermoelectric generator
Radioisotope heater unit
Radioisotope piezoelectric generator
Radioisotope rocket
Nuclear thermal rocket
Nuclear pulse propulsion
Nuclear electric rocket
"""

docs = [
    Document(
        page_content=split, 
        metadata={"source": "https://en.wikipedia.org/wiki/Nuclear_power_in_space"}
    ) 
    for split in text.split("\n\n")
]

In [72]:
print(map_reduce.invoke(docs[0:1], config={"max_concurrency": 5}))

 Here is a combined summary:

Nuclear power has several uses in outer space applications, the most common being radioisotope thermoelectric generators. These generators use heat from radioactive decay to generate electricity. Overall, nuclear power has proven useful for providing electricity, heat, and power for scientific instruments in the extreme environment of space.
